# ExecutorService

- 쓰레드 풀을 이용해 비동기적으로 작업을 실행하고 관리
- 쓰레드 풀을 이용해 자원을 효율적으로 관리
- 별도의 쓰레드를 생성하고 관리하지 않아도 되어서 코드를 간결하게 유지할 수 있음

method
- execute: Runnable 인터페이스를 구현한 작업을 쓰레드풀에서 비동기적으로 실행
- submit: Callable 인터페이스를 구현한 작업을 쓰레드풀에서 비동기적으로 실행, 해당 작업 결과를 Future<T>로 반환
- shutdown: ExecutorService 를 종료, 더이상 task를 받지 않음                                                                                 



In [1]:
import java.util.concurrent.ExecutorService
import java.util.concurrent.Executors

Executors.newSingleThreadExecutor() // 단일 쓰레드로 구성된 스레드풀을 생성

Executors.newFixedThreadPool(1) // 고정된 크기의 쓰레드 풀 생성

Executors.newCachedThreadPool() // 사용 가능한 쓰레드가 없다면 새로 생성, 있다면 재사용. 쓰레드가 일정시간 사용되지 않으면 회수함

Executors.newScheduledThreadPool(1) // 스케줄링 기능을 갖춘 코정 크기의 쓰레드풀 생성. 주기적이거나 지연이 발생하는 작업을 실행

Executors.newWorkStealingPool() // work streal 알고지름을 사용하는 ForkJoinPool 생성

java.util.concurrent.ThreadPoolExecutor@7f9b2075[Running, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0]

# Future

- 비동기 적인 작업 수행
- 작업이 완료되면 결과를 반환


In [3]:
import java.util.concurrent.Executor
import java.util.concurrent.Executors
import java.util.concurrent.Future

fun getFuture(): Future<Int> {
    val executor = Executors.newSingleThreadExecutor()
    try {
        return executor.submit<Int> { 1 }
    } finally {
        executor.shutdown()
    }
}


fun getFutureCompleteAfter1s(): Future<Int> {
    val executor = Executors.newSingleThreadExecutor()
    try {
        return executor.submit<Int> {
            Thread.sleep(1000)
            1
        }
    } finally {
        executor.shutdown()
    }
}

In [5]:
val future = getFuture()
println(future.isDone) // task가 완료되었다면 true
println(future.isCancelled) // task가 명시적으로 취소되었다면 true

false
false


In [7]:
val result = future.get() // 결과를 구할때까지 thread를 block
println(future.isDone)
println(future.isCancelled)
println(result)

true
false
1


In [9]:
import java.util.concurrent.TimeUnit

val future = getFutureCompleteAfter1s()
val result = future.get(500L, TimeUnit.MICROSECONDS) // 500ms 타임아웃 설정 -> 에러 발생

null
java.util.concurrent.TimeoutException
	at java.base/java.util.concurrent.FutureTask.get(FutureTask.java:204)
	at Line_9_jupyter.<init>(Line_9.jupyter.kts:4)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at o

In [10]:
val future = getFuture()
val isCancllled = future.cancel(true) // future의 작업을 취소, 취소할 수 없는 상황이면 false 반환
// parameter(mayInterruptIfRunning)가 false이면 시작하지 않은 작업에 대해서만 취소한다

println(isCancllled)
println(future.isDone)
println(future.isCancelled)



true
true
true


## Future 인터페이스의 한계
- cancel을 제외하고 외부에서 future를 컨트롤 할 수 없음
- 반환된 결과를 get()으로 접근하기에 비동기 처리가 어려움
- 완료나 에러가 발생했는지 구분이 어려움